In [2]:
%matplotlib inline 

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import sklearn
import datetime
import array 
import math

import shapefile as sf
from matplotlib.patches import Polygon
import mpld3

from censusgeocode import CensusGeocode
from matplotlib.collections import PatchCollection

In [2]:
def whichshape(lat,lon,path):
    sr = sf.Reader(path)
    shapes = sr.shapes()
    for i in range(len(shapes)):
        if inshape(lat,lon,shapes[i]):
            return i

In [3]:
def inshape(ptlat,ptlon,shape):
    # does not behave well near poles!!!
    pts = shape.points
    lat=[]
    lon=[]
    for pt in pts:
        lat.append(pt[1])
        lon.append(pt[0])
    lon_max=max(lon)
    lon_min=min(lon)
    lat_max=max(lat)
    lat_min=min(lat)
    
    if lon_min < -170 and lon_max > 170:
        for i in range(len(pts)):
            if pts[i][0]<0:
                pts[i][0] = 360+pts[i]
    
    if (ptlat > lat_max or ptlat < lat_min or
        ptlon > lon_max or ptlon < lon_min):
        return 0
    else:
        count = 0
        for i in range(len(pts)-1):
            line=[pts[i][0],pts[i][1],
                  pts[i+1][0],pts[i+1][1]]
                  
            if ((line[1] < ptlat and line[3] > ptlat) or 
                (line[1] > ptlat and line[3] < ptlat)):
               frac = (line[1]-ptlat)/(line[1]-line[3])
               if line[0]+frac*(line[2]-line[0]) > ptlon:
                    count+=1
        
        line=[pts[-1][0],pts[0][1],
              pts[-1][0],pts[0][1]]                  
        if line[1] < ptlat and line[3] > ptlat:
           frac = (line[1]-ptlat)/(line[1]-line[3])
           if line[0]+frac*(line[2]-line[0]) > ptlon:
                count+=1

    return count%2         

### 311 Request files

In [4]:
requests = pd.read_csv('DataforInsight.csv')

/Users/kschles/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,17,19,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
requests[['Problem Category', 'Case Number']].groupby('Problem Category').count().sort_values(by='Case Number')

,Case Number
Problem Category,
Over Irrigation,1
Parking Meter,34
Damaged Guardrail,61
Street Flooded,100
Dead Animal,122
Faded striping,193
Street Sweeping,258
Storm Drain,274
Tree Hazard,752


Isolate requests of interest

In [6]:
graffiti = requests.loc[np.where(requests['Problem Category']=='Graffiti')[0]]
graffiti.reset_index(drop=True, inplace=True)

dump = requests.loc[np.where(requests['Problem Category']=='Litter/Dumping')[0]]
dump.reset_index(drop=True, inplace=True)

lighting = requests.loc[np.where(requests['Problem Category']=='Street Light')[0]]
lighting.reset_index(drop=True, inplace=True)

### Extract Census linking information for request locations 

In [7]:
cg = CensusGeocode()

In [542]:
requests['block']=None
requests['tract']=None
requests['geoid']=None
requests['blockgroup']=None

In [613]:
start = datetime.datetime.now()

for i in range(0,len(requests)): 
    if ((requests.loc[i,'Geolocation (Longitude)']!=0.0) 
        & (np.isnan(requests.loc[i,'Geolocation (Longitude)'])==False)) :
        temp1 = cg.coordinates(x=requests.loc[i,'Geolocation (Longitude)'], 
                              y=requests.loc[i, 'Geolocation (Latitude)'])
        if (len(temp1[0]['2010 Census Blocks'])>0):
            if ('status' not in temp1[0]['2010 Census Blocks'][0].keys()[0]): 
                temp = temp1[0]['2010 Census Blocks'][0]
                if (temp is not None):
                    requests.loc[i, 'block'] = temp['BLOCK']
                    requests.loc[i, 'tract'] = temp['TRACT']
                    requests.loc[i, 'geoid'] = temp['GEOID']
                    requests.loc[i, 'blockgroup'] = temp['BLKGRP']

                if (i % 100 == 0): 
                    finish = datetime.datetime.now()
                    print i, finish-start
                    start = datetime.datetime.now()



13200 0:01:17.631727
13300 0:01:02.481330
13400 0:01:12.625485
13500 0:01:18.448509
13600 0:01:03.096039
13700 0:01:17.490964
13800 0:01:22.407940
13900 0:01:02.041980
14000 0:01:08.443168
14100 0:01:09.428046
14200 0:01:07.492265
14300 0:00:56.761278
14400 0:01:13.255247
14500 0:01:12.487709
14600 0:01:24.316358
14700 0:01:07.770941
14800 0:01:24.572345
14900 0:01:09.458441
15000 0:01:03.182257
15100 0:01:02.565870
15200 0:01:02.951913
15300 0:01:28.528602
15400 0:00:57.778108
15500 0:00:57.970770
15600 0:01:21.884530
15700 0:01:18.255734
15800 0:01:03.000292
15900 0:01:08.006805
16000 0:01:01.867869
16100 0:01:02.033723
16200 0:01:14.390271
16300 0:01:02.519959
16400 0:01:18.055804
16500 0:01:07.159415
16600 0:01:07.483389
16700 0:01:17.849506
16800 0:01:06.354572
16900 0:01:25.211410
17000 0:01:09.805353
17100 0:01:18.925729
17200 0:01:03.003951
17300 0:01:08.916376
17400 0:01:08.429254
17500 0:01:22.755352
17600 0:01:12.540263
17700 0:01:08.728271
17800 0:01:13.355258
17900 0:01:20

In [3]:
# Once all the geocodes are done, output the file as requests_processed_start.csv: 
#requests.to_csv('output_files/requests_processed_start.csv', index=False)
requests = pd.read_csv('output_files/requests_processed_start.csv')

/Users/kschles/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,17,19,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Remove Duplicate requests
requests.columns.values

requests_unique = requests.loc[np.where(requests['Duplicate Verified']==0.0)[0]]
requests_unique.reset_index(drop=True, inplace=True)

In [5]:
# Count up graffiti reports
requests_unique['graffiti_count'] = 0.0

requests_unique.loc[np.where(requests_unique['Problem Category']=='Graffiti')[0], 
                    'graffiti_count'] = 1.0

# Count up dumping reports
requests_unique['dump_count'] = 0.0

requests_unique.loc[np.where(requests_unique['Problem Category']=='Litter/Dumping')[0], 
                    'dump_count'] = 1.0

# Count up street light reports
requests_unique['lighting_count'] = 0.0

requests_unique.loc[np.where(requests_unique['Problem Category']=='Street Light')[0], 
                    'lighting_count'] = 1.0

For graffiti and dumping reports, calculate the distance from the nearest streetlight.

In [6]:
sl_record = pd.read_csv('output_files/streetlight_latlon.csv')

For each event, I want to calculate the minimum distance from a street light. 
Only doing this for graffiti and dumping.

In [7]:
requests_unique['sl_mindist'] = None

In [25]:
for i in range(0,len(requests_unique)): 
    if ((requests_unique.loc[i,'Problem Category']=='Graffiti') 
        | (requests_unique.loc[i, 'Problem Category']=='Litter/Dumping')): 
        lat1=requests_unique.loc[i, 'Geolocation (Latitude)']
        lon1=requests_unique.loc[i, 'Geolocation (Longitude)']

        R = 6371 * 3280.84 # feet

        dLat = np.radians(np.array(np.array(sl_record['lat']) - lat1).astype(float))
        dLon = np.radians(np.array(np.array(sl_record['lon']) - lon1).astype(float))
        lat_source = np.radians(lat1)
        lat_comp = np.radians(np.array(sl_record['lat']).astype(float))


        a = np.sin(dLat/2.0) * np.sin(dLat/2.0) + np.sin(dLon/2.0) * np.sin(dLon/2.0) * np.cos(lat_source) * np.cos(lat_comp) 
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c

        requests_unique.loc[i, 'sl_mindist'] = float(d[np.argmin(d)])

In [26]:
requests_unique['sl_mindist'] = requests_unique['sl_mindist'].astype(float)

Output the file that keeps track of the individual request informations. 

In [30]:
#requests_unique.to_csv('output_files/requests_processed_0917.csv', index=False)

In [8]:
requests_unique = pd.read_csv('output_files/requests_processed_0917.csv')

/Users/kschles/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17,19,43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Now we have things organised for individual requests. We also want to organise the requests by their block groups. 

First, organise the individual requests by block group.

In [9]:
requests_unique['count'] = 1.0
total_reports = requests_unique[['tract', 'blockgroup', 'count', 
                                 'graffiti_count','dump_count', 
                                 'lighting_count']].groupby(['tract', 'blockgroup'], as_index=False).agg('sum')

We want to break it up by how the 311 reports came in as well. 

In [10]:
mobile = requests_unique.loc[np.where(requests_unique['Case Origin']=='Mobile')[0], ['tract','blockgroup',
                                                               'Case Origin']]
web = requests_unique.loc[np.where(requests_unique['Case Origin']=='Web')[0], ['tract','blockgroup',
                                                               'Case Origin']]
phone = requests_unique.loc[np.where(requests_unique['Case Origin']=='Phone')[0], ['tract','blockgroup',
                                                               'Case Origin']]

In [11]:
mobile_counts = mobile.groupby(['tract', 'blockgroup'], as_index=False).agg('count')
mobile_counts.rename(columns={'Case Origin': 'mobile_counts'}, inplace=True)

web_counts = web.groupby(['tract', 'blockgroup'], as_index=False).agg('count')
web_counts.rename(columns={'Case Origin': 'web_counts'}, inplace=True)

phone_counts = phone.groupby(['tract', 'blockgroup'], as_index=False).agg('count')
phone_counts.rename(columns={'Case Origin': 'phone_counts'}, inplace=True)

In [12]:
temp1 = pd.merge(total_reports, mobile_counts, on=['tract', 'blockgroup'], how='left')
temp2 = pd.merge(temp1, web_counts, on=['tract', 'blockgroup'], how='left')
total_reports = pd.merge(temp2, phone_counts, on=['tract', 'blockgroup'], how='left')

Next, we want to combine with the street light information. 

In [14]:
# First, calculate the mean distance from a street light 
# for different events organised by blockgroup. 
# Mean sl_dist for each census track, block group 
graffiti_distance = requests_unique.loc[np.where(requests_unique['Problem Category']=='Graffiti')[0],
                                        ['tract', 'blockgroup', 'sl_mindist']].groupby(['tract', 
                                                                                        'blockgroup'], as_index=False).agg('mean')


dump_distance = requests_unique.loc[np.where(requests_unique['Problem Category']=='Litter/Dumping')[0],
                                        ['tract', 'blockgroup', 'sl_mindist']].groupby(['tract', 
                                                                                        'blockgroup'], as_index=False).agg('mean')

#response_time = requests_unique.loc[np.where(requests_unique['Problem Category']=='Graffiti')[0],
#                                        ['tract', 'blockgroup', 'Age (Days)', 'count']].groupby(['tract', 
#                                                                                        'blockgroup'], as_index=False).agg({'count' :'sum','sl_mindist': 'mean'})

graffiti_distance.rename(columns={'sl_mindist': 'gsl_meandist'}, inplace=True)
dump_distance.rename(columns={'sl_mindist': 'dsl_meandist'}, inplace=True)

In [15]:
# First, calculate the mean distance from a street light 
# for different events organised by blockgroup. 
# Mean sl_dist for each census track, block group 
graffiti_distance1 = requests_unique.loc[np.where(requests_unique['Problem Category']=='Graffiti')[0],
                                        ['tract', 'blockgroup', 'sl_mindist', 'count']].groupby(['tract', 
                                                                                        'blockgroup'], as_index=False).agg({'count' :'sum','sl_mindist': 'mean'})

dump_distance1 = requests_unique.loc[np.where(requests_unique['Problem Category']=='Litter/Dumping')[0],
                                        ['tract', 'blockgroup', 'sl_mindist', 'count']].groupby(['tract', 
                                                                                        'blockgroup'], as_index=False).agg({'count' :'sum','sl_mindist': 'mean'})

#response_time1 = requests_unique.loc[np.where(requests_unique['Problem Category']=='Graffiti')[0],
#                                        ['tract', 'blockgroup', 'Age (Days)', 'count']].groupby(['tract', 
#                                                                                        'blockgroup'], as_index=False).agg('std')

graffiti_distance1.rename(columns={'sl_mindist': 'gsl_stddist'}, inplace=True)
dump_distance1.rename(columns={'sl_mindist': 'dsl_stddist'}, inplace=True)

In [16]:
# First, calculate the mean distance from a street light 
# for different events organised by blockgroup. 
# Mean sl_dist for each census track, block group 
graffiti_distance2 = requests_unique.loc[np.where(requests_unique['Problem Category']=='Graffiti')[0],
                                        ['tract', 'blockgroup', 'sl_mindist']].groupby(['tract', 
                                                                                        'blockgroup'], as_index=False).agg('median')

dump_distance2 = requests_unique.loc[np.where(requests_unique['Problem Category']=='Litter/Dumping')[0],
                                        ['tract', 'blockgroup', 'sl_mindist']].groupby(['tract', 
                                                                                        'blockgroup'], as_index=False).agg('median')

response_time2 = requests_unique.loc[np.where(requests_unique['Problem Category']=='Graffiti')[0],
                                        ['tract', 'blockgroup', 'Age (Days)']].groupby(['tract', 
                                                                                        'blockgroup'], as_index=False).agg('median')

graffiti_distance2.rename(columns={'sl_mindist': 'gsl_mediandist'}, inplace=True)
dump_distance2.rename(columns={'sl_mindist': 'dsl_mediandist'}, inplace=True)

In [17]:
# Merge with minimum distance information: 
temp1 = pd.merge(total_reports, graffiti_distance, on=['tract', 'blockgroup'], 
                 how='left')
temp2 = pd.merge(temp1, graffiti_distance1[['tract', 'blockgroup', 'gsl_stddist']], on=['tract', 'blockgroup'], 
                 how='left')
temp3 = pd.merge(temp2, graffiti_distance2, on=['tract', 'blockgroup'], 
                 how='left')

temp4 = pd.merge(temp3, dump_distance, on=['tract', 'blockgroup'], 
                 how='left')
temp5 = pd.merge(temp4, dump_distance1[['tract', 'blockgroup', 'dsl_stddist']], on=['tract', 'blockgroup'], 
                 how='left')
total_reports = pd.merge(temp5, dump_distance2, on=['tract', 'blockgroup'], 
                         how='left')

In [18]:
total_reports.columns.values

array(['tract', 'blockgroup', 'count', 'graffiti_count', 'dump_count',
       'lighting_count', 'mobile_counts', 'web_counts', 'phone_counts',
       'gsl_meandist', 'gsl_stddist', 'gsl_mediandist', 'dsl_meandist',
       'dsl_stddist', 'dsl_mediandist'], dtype=object)

In [19]:
# Merge with information about street light density and street light counts
sl_info = pd.read_csv('output_files/streetlight_density.csv')

In [20]:
sl_info['tract'] = sl_info['tract'].astype(float)
sl_info['blockgroup'] = sl_info['blockgroup'].astype(float)
sl_info.rename(columns={'count': 'sl_count'}, inplace=True)

In [21]:
reports_sl = pd.merge(total_reports, sl_info, on=['tract', 'blockgroup'], how='left')

#### Now I have all the reports broken down by block group. However, not all of them are in San Diego county. Need to trim things down appropriately.

In [22]:
# Blockgroup shape file
reader = sf.Reader('shapefiles/tl_2016_06_bg/tl_2016_06_bg.shp')
bg_info = pd.DataFrame(reader.records())
shapes = reader.shapes()

# the San Diego county shapefile records
sdcounty = bg_info.loc[np.where((bg_info[1]=='073') & (bg_info[3]!='0'))[0]]
sdcounty_index = np.where((bg_info[1]=='073') & (bg_info[3]!='0'))[0]

In [23]:
len(bg_info), len(sdcounty)

(23212, 1794)

In [24]:
sdcounty['tract'] = sdcounty[2].astype(float)
sdcounty['blockgroup'] = sdcounty[3].astype(float)

In [25]:
sdcounty_only_requests = pd.merge(sdcounty[['tract','blockgroup']], reports_sl, 
                                  on=['tract', 'blockgroup'], how='left')

In [26]:
# Output as a CSV
#sdcounty_only_requests.to_csv('output_files/sdcounty_only_requests_0917.csv', index=False)

#### Now I have all the info I need sorted by block group

I now need to combine with some census information to get the rest of the detailed columns I need. 

In [27]:
## Want to match report numbers with Census information about the total people in a given area

filenames='aff/block_groups/*with_ann.csv'
test=glob.glob(filenames)

all_of_interest = [33, 4]

for i in all_of_interest: 
    temp = pd.read_csv(test[i], header=1, low_memory=False)
    print i, i, i 
    if (i==33): 
        census = temp.copy()
    if (i!=33): 
        combo = pd.merge(census, temp, on='Id', how='left', suffixes=('', '_' + str(i)))
        census = combo.copy()
        census.reset_index(drop=True)

33 33 33
4 4 4


In [28]:
census_trimmed = census[['GEOGRAPHIC AREA CODES - Census Tract',
                         'GEOGRAPHIC AREA CODES - Block Group',
                         'Estimate; Total']].copy()

census_trimmed.rename(columns={'GEOGRAPHIC AREA CODES - Census Tract': 'tract', 
                               'GEOGRAPHIC AREA CODES - Block Group': 'blockgroup'}, 
                      inplace=True)

In [29]:
# Merge report numbers with population estimates
total_reports_demo = pd.merge(sdcounty_only_requests, 
                              census_trimmed, on=['tract', 'blockgroup'], how='left')

In [30]:
# Currently set to NaN when there are no counts. Change to 0. 
total_reports_demo.loc[np.where(np.isnan(total_reports_demo['count'])==True)[0], 'count'] = 0.0

In [31]:
# Reports per capita
total_reports_demo['rep_per_cap'] = (total_reports_demo['count'] 
                                     / total_reports_demo['Estimate; Total'])

# Percentage of reports in each category
total_reports_demo['graffiti_per'] = (total_reports_demo['graffiti_count'] 
                                      / total_reports_demo['count'])
total_reports_demo['dump_per'] = (total_reports_demo['dump_count'] 
                                  / total_reports_demo['count'])
total_reports_demo['lighting_per'] = (total_reports_demo['lighting_count'] 
                                      / total_reports_demo['count'])

In [32]:
total_reports_demo.rename(columns={'Estimate; Total': 'total_pop'}, inplace=True)

#### I have processed all of the information county-wide. But now I want to identify items that are in San Diego city proper.

In [50]:
# Read in Jeff's list of census tracts in SD city proper 
sd_city_proper = pd.read_csv('SD_city_census_tracts.txt')

sd_city_proper['tract'] = sd_city_proper['tract'].astype(float)

In [51]:
# Read in the edited list i made of block groups that are in municipal-range census tracts, 
# but not within the city itself
sd_bg_edited = pd.read_csv('tract_blockgroups_to_ignore.csv')
sd_bg_edited['blockgroup'] = sd_bg_edited['blockgroup'].astype(float)
sd_bg_edited['tract'] = sd_bg_edited['tract'].astype(float)

In [70]:
total_reports_demo['tract_in_city'] = 0.0

In [71]:
for i in range(0,len(total_reports_demo)): 
    temp_tract = total_reports_demo.loc[i, 'tract']
    temp_bg = total_reports_demo.loc[i, 'blockgroup']
    if (len(np.where(sd_city_proper['tract']==temp_tract)[0])>0):
        if (len(np.where((sd_bg_edited['tract']==temp_tract) 
                         & (sd_bg_edited['blockgroup']==temp_bg))[0])==0):        
            total_reports_demo.loc[i, 'tract_in_city'] = 1.0

In [72]:
print 'In city proper:', len(np.where(total_reports_demo['tract_in_city']==1.0)[0]) 
print 'Outside city proper:', len(np.where(total_reports_demo['tract_in_city']==0.0)[0])

In city proper: 836
Outside city proper: 958


In [73]:
# Output the reports organised by block group 
total_reports_demo.to_csv('output_files/total_reports_demo_0917.csv', inplace=True)

In [62]:
total_reports_demo.columns.values

array(['tract', 'blockgroup', 'count', 'graffiti_count', 'dump_count',
       'lighting_count', 'gsl_meandist', 'gsl_stddist', 'gsl_mediandist',
       'dsl_meandist', 'dsl_stddist', 'dsl_mediandist', 'sl_count',
       'landarea', 'cent_lat', 'cent_lon', 'sl_density', 'total_pop',
       'rep_per_cap', 'graffiti_per', 'dump_per', 'lighting_per',
       'tract_in_city'], dtype=object)

#### Using Tim's code instead of geocoder:

Sometimes Census geocoder is down. This allows you to match each object with a shapefile region instead. It's a bit slower. Trim shapefile down to San Diego County

In [10]:
r = sf.Reader('shapefiles/tl_2016_06_bg/tl_2016_06_bg.shp')
w = sf.Writer(shapeType=sf.POLYGON)

In [11]:
# Copy the fields to the writer
w.fields = list(r.fields)

In [12]:
# Grab the geometry and records from all features 
# with the correct county name 
selection = [] 
for rec in enumerate(r.records()):
    if ((rec[1][1]=='073') & (rec[1][3]!='0')):
          selection.append(rec) 
# Add the geometry and records to the writer

for rec in selection:
    w._shapes.append(r.shape(rec[0]))
    w.records.append(rec[1])
# Save the new shapefile
w.save("shapefiles/sandiego_county_blockgroups") 

Match each 311 report with it's associated block group

In [62]:
temporary = lighting 

temporary['block_group']=None
temporary['tract']=None
temporary['geoid']=None

reading = sf.Reader('shapefiles/sandiego_county_blockgroups.shp')

start = datetime.datetime.now()

for i in range(0,len(lighting)): 
    if (temporary.loc[i,'Geolocation (Longitude)']!=0.0) :
        temp = whichshape(temporary.loc[i,'Geolocation (Latitude)'], 
                          temporary.loc[i, 'Geolocation (Longitude)'], 
                          'shapefiles/sandiego_county_blockgroups.shp')
        if (temp is not None):
            temporary.loc[i, 'block_group'] = reading.records()[temp][3]
            temporary.loc[i, 'tract'] = reading.records()[temp][2]
            temporary.loc[i, 'geoid'] = reading.records()[temp][4]
        
    if (i % 100 == 0): 
        finish = datetime.datetime.now()
        print i, finish-start
        start = datetime.datetime.now()



0 0:00:00.920797
100 0:01:41.484810
200 0:01:38.798607
300 0:01:41.140341
400 0:01:44.568110
500 0:01:42.041521
600 0:01:42.453154
700 0:01:42.134142
800 0:01:43.106106
900 0:01:42.113004
1000 0:01:41.735002
1100 0:01:41.449113
1200 0:01:40.819706
1300 0:01:43.134757
